In [1]:
import requests
import polyline
import csv
import folium
import os
import numpy as np
import pandas as pd
import torch
import cv2
import re
from PIL import Image

# Gete Routes

The following section is used to generate navigation routes from two sets of points. The code uses the OSRM API to generate the the navagation route. And the generated route is represented by coordinates (lat/lng) and are saved into csv files. The saved coordinates will be used to download streetview images from

In [2]:
def get_osrm_route(start, end, profile="driving"):
    """
    Get the route from OSRM and extract the GPS coordinates with a preference for local routes.

    :param start: Tuple containing the starting latitude and longitude (e.g., (13.388860, 52.517037))
    :param end: Tuple containing the ending latitude and longitude (e.g., (13.397634, 52.529407))
    :param profile: Routing profile, e.g., "driving", "walking", "cycling"
    :return: List of tuples containing the GPS coordinates of the route
    """
    # OSRM base URL for routing
    osrm_base_url = f"http://router.project-osrm.org/route/v1/{profile}/{start[1]},{start[0]};{end[1]},{end[0]}"
    
    # Additional parameters for the route request
    params = {
        'overview': 'full',        # Get the full route geometry
        'geometries': 'polyline',  # Get polyline format for the route
        'steps': 'false',          # Do not include detailed step-by-step information
        #'exclude': 'motorway'      # Avoid motorways to favor local routes
    }
    
    # Make the request to OSRM API
    response = requests.get(osrm_base_url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        
        # Extract the polyline of the route
        polyline_str = data['routes'][0]['geometry']
        
        # Decode the polyline into GPS coordinates (lat, lon)
        coordinates = polyline.decode(polyline_str)
        return coordinates
    else:
        print(f"Error: Unable to fetch route. Status code: {response.status_code}")
        return None

def save_coordinates_to_csv(coordinates, filename="route_coordinates.csv"):
    """
    Save the GPS coordinates to a CSV file.

    :param coordinates: List of tuples containing GPS coordinates (latitude, longitude)
    :param filename: Name of the CSV file to save the coordinates
    """
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write header
        writer.writerow(['Latitude', 'Longitude'])
        
        # Write each coordinate to the CSV
        for coord in coordinates:
            writer.writerow(coord)

    print(f"Coordinates have been saved to {filename}")

# Generate Routes and Get Points
Use start coordinates and end coordinates of each point within the city to generate coordinates

# Download Streeview Images
Based on the previously generated navigation routes, download streetview images per route coordinates and save them in a folder, also create a csv file to store the information of each image

In [3]:
# Download a single Street View image and fetch minimal metadata
def download_image_with_metadata(lat, lon, api_key, folder, image_name, csv_writer, heading=0, fov=40, pitch=0):
    """
    Downloads a Street View image and retrieves minimal metadata (coordinates and timestamp).
    Writes metadata directly to the CSV file.
    """
    url = "https://maps.googleapis.com/maps/api/streetview"
    meta_url = "https://maps.googleapis.com/maps/api/streetview/metadata"
    params = {
        'size': '640x640',
        'location': f'{lat},{lon}',
        'heading': heading,
        'fov': fov,
        'pitch': pitch,
        'key': api_key
    }
    
    # Fetch metadata
    meta_response = requests.get(meta_url, params=params)
    if meta_response.status_code == 200:
        metadata = meta_response.json()
        timestamp = metadata.get('date', 'Unknown')  # Fetch timestamp if available
    else:
        timestamp = 'Unknown'
    
    # Download image
    response = requests.get(url, params=params, stream=True)
    if response.status_code == 200:
        os.makedirs(folder, exist_ok=True)
        with open(os.path.join(folder, f"{image_name}.jpg"), 'wb') as f:
            f.write(response.content)
        csv_writer.writerow({
            'image_name': image_name,  # Save without .jpg
            'latitude': lat,
            'longitude': lon,
            'timestamp': timestamp
        })
    else:
        pass  # Skipping print to keep the function silent

# Process a CSV file and collect metadata
def process_csv_with_metadata(csv_path, api_key, folder, csv_writer, max_images, start_index):
    """
    Processes a CSV file to download images and collect metadata (coordinates and timestamp).
    Stops when max_images is reached.
    """
    coords = pd.read_csv(csv_path)
    images_downloaded = 0
    for i, (lat, lon) in enumerate(zip(coords['Latitude'], coords['Longitude']), start=start_index + 1):
        if images_downloaded >= max_images:
            break
        download_image_with_metadata(lat, lon, api_key, folder, str(i), csv_writer)  # Image name is str(i)
        images_downloaded += 1
    return images_downloaded

# Main function for a city with metadata collection
def download_city_images_with_metadata(city, api_key, image_quantity, route_folder="routes", output_base="streetview_images"):
    """
    Downloads Street View images for a city and saves metadata (coordinates and timestamp).
    Stops when the total number of downloaded images reaches image_quantity.
    """
    route_path = os.path.join(route_folder, city)
    output_folder = os.path.join(output_base, city)
    metadata_csv_path = os.path.join(output_folder, f"{city}_metadata.csv")
    downloaded_count = 0
    start_index = 0

    # Create output folder and metadata CSV
    os.makedirs(output_folder, exist_ok=True)
    with open(metadata_csv_path, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['image_name', 'latitude', 'longitude', 'timestamp'])
        writer.writeheader()

        # Process all route files
        for file in sorted(os.listdir(route_path)):
            if file.endswith(".csv"):
                remaining_images = image_quantity - downloaded_count
                if remaining_images <= 0:
                    break
                images_downloaded = process_csv_with_metadata(
                    os.path.join(route_path, file),
                    api_key,
                    output_folder,
                    writer,
                    remaining_images,
                    start_index
                )
                downloaded_count += images_downloaded
                start_index += images_downloaded


In [58]:
def download_image_with_metadata(lat, lon, api_key, folder, image_name, csv_writer, heading=0, fov=40, pitch=0):
    """
    Downloads a Street View image and retrieves minimal metadata (coordinates and timestamp).
    Writes metadata directly to the CSV file.
    """
    url = "https://maps.googleapis.com/maps/api/streetview"
    meta_url = "https://maps.googleapis.com/maps/api/streetview/metadata"

    params = {
        'size': '640x640',
        'location': f'{lat},{lon}',
        'heading': heading,
        'fov': fov,
        'pitch': pitch,
        'key': api_key
    }

    # Fetch metadata
    meta_response = requests.get(meta_url, params=params)
    if meta_response.status_code == 200:
        metadata = meta_response.json()
        if metadata.get('status') == 'OK':
            timestamp = metadata.get('date', 'Unknown')
        else:
            timestamp = 'Unknown'
    else:
        timestamp = 'Unknown'

    # Download image
    response = requests.get(url, params=params, stream=True)
    if response.status_code == 200:
        os.makedirs(folder, exist_ok=True)
        image_path = os.path.join(folder, f"{image_name}.jpg")
        with open(image_path, 'wb') as f:
            f.write(response.content)

        # Write minimal metadata to CSV
        csv_writer.writerow({
            'image_name': image_name,
            'latitude': lat,
            'longitude': lon,
            'timestamp': timestamp
        })
    else:
        # If needed, log or handle error responses here
        pass


def download_city_images_with_metadata(city, api_key, image_quantity,
                                       route_folder="routes", 
                                       output_base="streetview_images"):
    """
    Downloads Street View images for a city and saves metadata (coordinates and timestamp).
    If there are already downloaded images, it continues from where it left off.
    For example, if there are already 100 images, and you ask for 10 more, 
    it downloads from 101 to 110.

    1. Reads all CSVs in the city's route folder, concatenates them into a single DataFrame.
    2. Checks how many images are already downloaded (via existing metadata CSV or .jpg files).
    3. Skips that many coordinate rows, then continues download for 'image_quantity' additional images.
    """

    route_path = os.path.join(route_folder, city)
    output_folder = os.path.join(output_base, city)
    metadata_csv_path = os.path.join(output_folder, f"{city}_metadata.csv")
    
    # ---- 1. Concatenate all CSV route files into one DataFrame ----
    all_coords = []
    for file in sorted(os.listdir(route_path)):
        if file.endswith(".csv"):
            csv_path = os.path.join(route_path, file)
            df_temp = pd.read_csv(csv_path)
            all_coords.append(df_temp)
    # If no CSVs are found, exit early
    if not all_coords:
        print(f"No CSV files found in {route_path}. Nothing to download.")
        return

    coords_df = pd.concat(all_coords, ignore_index=True)
    
    # Ensure 'Latitude' and 'Longitude' columns are present
    # (optional: adapt naming logic if your columns differ)
    if 'Latitude' not in coords_df.columns or 'Longitude' not in coords_df.columns:
        raise ValueError("Input CSV(s) must contain 'Latitude' and 'Longitude' columns.")

    # ---- 2. Determine how many images are already downloaded ----
    # Option A: Check existing metadata CSV if it exists
    existing_count = 0
    if os.path.exists(metadata_csv_path):
        with open(metadata_csv_path, 'r', encoding='utf-8') as f:
            # Subtract 1 for the header row if using DictWriter
            existing_count = sum(1 for _ in f) - 1  # lines minus header
        # Safety check: If the file is empty or only header, existing_count can go negative
        existing_count = max(existing_count, 0)
    else:
        # Option B: If there's no metadata CSV, check .jpg files
        # (You can skip this or keep it as a fallback.)
        if os.path.exists(output_folder):
            jpg_files = [f for f in os.listdir(output_folder) if f.lower().endswith('.jpg')]
            existing_count = len(jpg_files)

    # If existing_count >= total coords, everything is downloaded
    if existing_count >= len(coords_df):
        print(f"All {existing_count} coordinates have been downloaded already.")
        return

    # ---- 3. Set up for continuing download ----
    # The new images will start from existing_count+1 up to existing_count + image_quantity
    next_start_index = existing_count  # zero-based index in coords_df
    final_index = existing_count + image_quantity  # i.e., exclusive end

    # Adjust final_index if it goes beyond the data length
    final_index = min(final_index, len(coords_df))

    # If there's nothing to download, exit
    if final_index <= next_start_index:
        print(f"Nothing to download; existing_count = {existing_count}, image_quantity = {image_quantity}.")
        return

    # Subset the coords we need to process
    coords_to_download = coords_df.iloc[next_start_index:final_index].reset_index(drop=True)

    # Create output folder
    os.makedirs(output_folder, exist_ok=True)
    
    # ---- 4. Open metadata CSV for appending (or create if needed) ----
    # If the file already exists and has a header, we can open in "a" mode.
    # If it doesn't exist, we create it with a header.
    file_mode = 'a' if os.path.exists(metadata_csv_path) else 'w'
    has_header = os.path.exists(metadata_csv_path)

    with open(metadata_csv_path, file_mode, newline='', encoding='utf-8') as csvfile:
        fieldnames = ['image_name', 'latitude', 'longitude', 'timestamp']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if not has_header:
            writer.writeheader()

        # ---- 5. Download loop ----
        # The image names should continue from existing_count+1, existing_count+2, ...
        for i, row in enumerate(coords_to_download.itertuples(index=False), start=existing_count+1):
            lat = row.Latitude
            lon = row.Longitude

            # Example: image_name is just the integer i
            image_name = str(i)
            download_image_with_metadata(
                lat=lat,
                lon=lon,
                api_key=api_key,
                folder=output_folder,
                image_name=image_name,
                csv_writer=writer
            )

    total_downloaded = final_index - existing_count
    print(f"Finished downloading {total_downloaded} images for {city}. "
          f"Metadata appended to {metadata_csv_path}.")

In [52]:
import os
import csv
import requests
import pandas as pd

def download_image_with_metadata(
    lat, 
    lon, 
    api_key, 
    folder, 
    image_name, 
    csv_writer, 
    heading=0, 
    fov=40, 
    pitch=0
):
    """
    Downloads a Street View image and retrieves minimal metadata (coordinates and timestamp).
    Writes metadata directly to the CSV file.
    """
    # Street View endpoints
    base_url = "https://maps.googleapis.com/maps/api/streetview"
    meta_url = "https://maps.googleapis.com/maps/api/streetview/metadata"

    params = {
        'size': '640x640',
        'location': f'{lat},{lon}',
        'heading': heading,
        'fov': fov,
        'pitch': pitch,
        'key': api_key
    }

    # Fetch metadata
    meta_response = requests.get(meta_url, params=params)
    if meta_response.status_code == 200:
        metadata = meta_response.json()
        if metadata.get('status') == 'OK':
            timestamp = metadata.get('date', 'Unknown')
        else:
            timestamp = 'Unknown'
    else:
        timestamp = 'Unknown'

    # Download image
    response = requests.get(base_url, params=params, stream=True)
    if response.status_code == 200:
        os.makedirs(folder, exist_ok=True)
        image_path = os.path.join(folder, f"{image_name}.jpg")
        with open(image_path, 'wb') as f:
            f.write(response.content)

        # Write metadata to CSV
        csv_writer.writerow({
            'image_name': image_name,
            'latitude': lat,
            'longitude': lon,
            'timestamp': timestamp
        })
    else:
        # Optionally handle or log response.status_code != 200
        pass

def download_city_images_with_metadata(
    city, 
    api_key, 
    image_quantity,
    route_folder="routes", 
    output_base="streetview_images"
):
    """
    Downloads Street View images for a city and saves metadata (coordinates and timestamp).
    1. Reads all CSVs in the city's route folder and concatenates into a single DataFrame.
    2. Checks existing metadata CSV (if any) to:
        - find how many images (rows) are already there
        - build a set of (lat, lon) already downloaded, so we won't duplicate
    3. Continues numbering images from the existing count + 1, skipping lat/lon pairs already downloaded.
    4. Stops after 'image_quantity' new images have been downloaded, or if no more new coords are available.
    """

    route_path = os.path.join(route_folder, city)
    output_folder = os.path.join(output_base, city)
    metadata_csv_path = os.path.join(output_folder, f"{city}_metadata.csv")

    # ---- STEP 1: Concatenate all CSV route files into one DataFrame ----
    all_coords = []
    for file in sorted(os.listdir(route_path)):
        if file.endswith(".csv"):
            csv_path = os.path.join(route_path, file)
            temp_df = pd.read_csv(csv_path)
            all_coords.append(temp_df)

    if not all_coords:
        print(f"No CSV files found in {route_path}. Nothing to download.")
        return

    coords_df = pd.concat(all_coords, ignore_index=True)

    # Ensure columns are present
    if 'Latitude' not in coords_df.columns or 'Longitude' not in coords_df.columns:
        raise ValueError("CSV(s) must contain 'Latitude' and 'Longitude' columns.")

    # ---- STEP 2: Check existing metadata CSV to see what is already downloaded ----
    existing_count = 0
    existing_coords = set()  # store (lat, lon) pairs so we don't duplicate

    if os.path.exists(metadata_csv_path):
        # Read existing metadata into a DataFrame
        existing_df = pd.read_csv(metadata_csv_path)
        
        # Keep track of how many images have been downloaded
        existing_count = len(existing_df)

        # Build a set of already-downloaded (lat, lon) pairs
        # to avoid duplicating them
        for _, row in existing_df.iterrows():
            lat_lon_pair = (round(row['latitude'], 8), round(row['longitude'], 8))
            existing_coords.add(lat_lon_pair)
    else:
        # Alternatively, if there's no metadata CSV,
        # we could do a quick check for .jpg files.
        if os.path.exists(output_folder):
            jpg_files = [f for f in os.listdir(output_folder) if f.lower().endswith('.jpg')]
            existing_count = len(jpg_files)

    # Next new image index (e.g., if existing_count=100, start with 101)
    next_image_index = existing_count + 1

    # ---- STEP 3: Filter out coordinates already downloaded ----
    # We'll round lat/lon to 8 decimals to match the approach in the set.
    coords_df['lat_rounded'] = coords_df['Latitude'].round(8)
    coords_df['lon_rounded'] = coords_df['Longitude'].round(8)

    # Drop any rows whose lat/lon are in existing_coords
    not_already_downloaded = coords_df[
        ~coords_df.apply(
            lambda row: (row['lat_rounded'], row['lon_rounded']) in existing_coords, 
            axis=1
        )
    ]
    not_already_downloaded = not_already_downloaded.reset_index(drop=True)

    if not_already_downloaded.empty:
        print(
            f"All coordinates appear to be already downloaded (based on lat/lon). "
            f"Existing count: {existing_count}"
        )
        return

    # ---- STEP 4: Prepare the CSV for writing/appending ----
    os.makedirs(output_folder, exist_ok=True)

    # Decide whether we're appending or creating
    file_mode = 'a' if os.path.exists(metadata_csv_path) else 'w'
    has_header = os.path.exists(metadata_csv_path)

    with open(metadata_csv_path, file_mode, newline='', encoding='utf-8') as csvfile:
        fieldnames = ['image_name', 'latitude', 'longitude', 'timestamp']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if not has_header:
            writer.writeheader()

        # ---- STEP 5: Download images up to image_quantity ----
        downloaded_count = 0

        for _, row in not_already_downloaded.iterrows():
            if downloaded_count >= image_quantity:
                break

            # Use next_image_index as the image_name (e.g. "101.jpg", "102.jpg", ...)
            image_name = str(next_image_index)

            # Download the image
            download_image_with_metadata(
                lat=row['Latitude'],
                lon=row['Longitude'],
                api_key=api_key,
                folder=output_folder,
                image_name=image_name,
                csv_writer=writer
            )

            downloaded_count += 1
            next_image_index += 1

    print(
        f"Finished downloading {downloaded_count} new images for {city}. "
        f"Total images now: {existing_count + downloaded_count}. "
        f"Metadata file: {metadata_csv_path}"
    )




# Crop Motorcycles By YoloV5 Default Model
Crop motorcycle images from original downloaded streetview images. To also get driver and passenger, multiple the size of the images accordingly

In [55]:
def load_yolov5_model(model_version='yolov5m'):
    """
    Load the YOLOv5 model from PyTorch Hub.
    """
    model = torch.hub.load('ultralytics/yolov5', model_version, pretrained=True)
    model.eval()
    return model

def crop_and_save(image_path, detections, output_folder, metadata, csv_writer, box_width_size=1.5, box_height_size=2.5):
    """
    Crop and save detected objects based on adjusted bounding boxes.
    Names cropped images using the original image name.
    Copies metadata from the parent image, adds file size, and writes it to the CSV file.
    """
    img = Image.open(image_path)
    img_width, img_height = img.size
    base_name = os.path.splitext(os.path.basename(image_path))[0]  # Extract base name without extension

    for i, det in enumerate(detections, start=1):
        x_min, y_min, x_max, y_max = map(int, det[:4])

        # Calculate the center and adjusted dimensions of the bounding box
        box_width = x_max - x_min
        box_height = y_max - y_min
        x_center = (x_min + x_max) / 2
        y_center = (y_min + y_max) / 2
        new_box_width = box_width * box_width_size
        new_box_height = box_height * box_height_size

        # Adjust coordinates and ensure they stay within image bounds
        new_x_min = int(max(x_center - new_box_width / 2, 0))
        new_x_max = int(min(x_center + new_box_width / 2, img_width))
        new_y_min = int(max(y_center - new_box_height / 2, 0))
        new_y_max = int(min(y_center + new_box_height / 2, img_height))

        # Crop the image
        cropped_img = img.crop((new_x_min, new_y_min, new_x_max, new_y_max))

        # Generate new file name (without extension)
        cropped_image_name = f"{base_name}_mtr_{i}"

        # Save the cropped image
        cropped_image_path = os.path.join(output_folder, f"{cropped_image_name}.jpg")
        cropped_img.save(cropped_image_path)

        # Calculate file size in KB
        file_size = os.path.getsize(cropped_image_path) / 1024  # File size in KB

        # Copy metadata from the parent and add to the CSV file
        cropped_metadata = metadata.copy()  # Copy metadata from parent
        cropped_metadata['parent_image'] = base_name
        cropped_metadata['cropped_image'] = cropped_image_name  # Save cropped image name without extension
        cropped_metadata['file_size'] = round(file_size, 2)  # Add file size (rounded to 2 decimals)
        cropped_metadata['driver'] = None
        cropped_metadata['passenger'] = None
        cropped_metadata['helmet'] = None
        cropped_metadata['driver_bbox'] = None
        cropped_metadata['passenger_bbox'] = None
        cropped_metadata['helmet_bbox'] = None
        csv_writer.writerow(cropped_metadata)

def process_single_image(model, image_path, output_folder, metadata, csv_writer, box_width_size=1.5, box_height_size=2.5, confidence_score=0.6):
    """
    Process a single image to detect and crop motorcycles.
    """
    results = model(image_path)
    detections = results.xyxy[0]

    # Filter for motorcycles (class 3) and confidence score
    motorcycle_detections = [det for det in detections if int(det[5]) == 3 and det[4] >= confidence_score]

    # If motorcycles are detected, crop and save them
    if motorcycle_detections:
        crop_and_save(image_path, motorcycle_detections, output_folder, metadata, csv_writer, box_width_size, box_height_size)

def process_motorcycle_images(image_folder, output_folder, metadata_csv_path, model_version='yolov5m', box_width_size=1.5, box_height_size=2.5, confidence_score=0.6):
    """
    Process all images in a folder to detect and crop motorcycles.
    Saves metadata for cropped images in a CSV file.
    """
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Load YOLOv5 model
    model = load_yolov5_model(model_version)

    # Read the metadata CSV file
    metadata_df = pd.read_csv(metadata_csv_path)
    metadata_mapping = {
        str(row['image_name']): {
            'latitude': row['latitude'],
            'longitude': row['longitude'],
            'timestamp': row['timestamp']
        }
        for _, row in metadata_df.iterrows()
    }

    # Create and open the CSV file for cropped image metadata
    cropped_metadata_csv_path = os.path.join(output_folder, f"cropped_metadata.csv")
    with open(cropped_metadata_csv_path, 'w', newline='') as csvfile:
        fieldnames = [
            'cropped_image', 'parent_image', 'latitude', 'longitude', 'timestamp', 'file_size',
            'driver', 'passenger', 'helmet',
            'driver_bbox', 'passenger_bbox', 'helmet_bbox'
        ]
        csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        csv_writer.writeheader()

        # Loop through all images in the folder
        for image_file in os.listdir(image_folder):
            if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(image_folder, image_file)

                # Extract metadata for the parent image
                base_name = os.path.splitext(image_file)[0]  # Get base name without extension
                metadata = metadata_mapping.get(base_name, {
                    'latitude': None, 'longitude': None, 'timestamp': None
                })

                # Process the image
                process_single_image(
                    model, image_path, output_folder, metadata, csv_writer, box_width_size, box_height_size, confidence_score
                )

def process_multiple_cities(target_cities, base_image_folder, base_output_folder, model_version='yolov5m', box_width_size=1.5, box_height_size=2.5, confidence_score=0.4):
    """
    Process multiple cities to detect and crop motorcycles from images.
    """
    for city in target_cities:
        image_folder = os.path.join(base_image_folder, city)
        output_folder = os.path.join(base_output_folder, city)
        
        # Find metadata CSV file in the city folder
        metadata_csv_path = os.path.join(image_folder, f"{city}_metadata.csv")
        if not os.path.exists(metadata_csv_path):
            raise FileNotFoundError(f"Metadata file not found for city: {city}")

        # Process the city
        process_motorcycle_images(image_folder, output_folder, metadata_csv_path, model_version, box_width_size, box_height_size, confidence_score)

# Below are the execution of the above script

## Generate Routes

In [4]:
#Example
start_coords = [(-15.741455963830575, -47.91516670553301), (-15.758588688653447, -47.935902941310545), (-15.80835598195138, -47.93121506981274), (-15.768112985633097, -47.939480679538825), (-15.779998798456628, -47.937525556241624)]
end_coords = [(-15.794993852686325, -47.84798782340603), (-15.837245547275772, -47.9202031492765), (-15.819358394228711, -47.86898900422858), (-15.768450300589505, -47.8567466800329), (-15.804926784989313, -47.87452982766739)]

target_city = 'Brasilia'

route_coords = []
for i in range(len(start_coords)):
    for j in range(len(end_coords)):
        route = get_osrm_route(start_coords[i], end_coords[j])
        route_coords.append(route)
# Create a folder to save the routes
if not os.path.exists(f"routes/{target_city}"):
        os.makedirs(f"routes/{target_city}") # Create a folder by the target city name
# Save the route to a CSV file in a folder iteratively
for route in route_coords:
    save_coordinates_to_csv(route, f"routes/{target_city}/route_coordinates_{route_coords.index(route)}.csv")

Coordinates have been saved to routes/Brasilia/route_coordinates_0.csv
Coordinates have been saved to routes/Brasilia/route_coordinates_1.csv
Coordinates have been saved to routes/Brasilia/route_coordinates_2.csv
Coordinates have been saved to routes/Brasilia/route_coordinates_3.csv
Coordinates have been saved to routes/Brasilia/route_coordinates_4.csv
Coordinates have been saved to routes/Brasilia/route_coordinates_5.csv
Coordinates have been saved to routes/Brasilia/route_coordinates_6.csv
Coordinates have been saved to routes/Brasilia/route_coordinates_7.csv
Coordinates have been saved to routes/Brasilia/route_coordinates_8.csv
Coordinates have been saved to routes/Brasilia/route_coordinates_9.csv
Coordinates have been saved to routes/Brasilia/route_coordinates_10.csv
Coordinates have been saved to routes/Brasilia/route_coordinates_11.csv
Coordinates have been saved to routes/Brasilia/route_coordinates_12.csv
Coordinates have been saved to routes/Brasilia/route_coordinates_13.csv
Co

## Download Images


In [23]:
#example
api_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'  
download_city_images_with_metadata('Pretoria', api_key, image_quantity=500)

## Crop Motorcycles 

In [30]:
#example
target_cities = ['Tunis', 'Sri_Jayawardenepura_Kotte', 'Colombo', 'Santo_Domingo' ,'Guatemala_City', 'Sucre', 'La_Paz', 'Santa_Cruz_de_la_Sierra', 'Pretoria', 'Cape_Town', 'Bloemfontein', 'Johannesburg']
base_image_folder = 'streetview_images'
base_output_folder = 'mtrcyc_images_cropped'
# Process all target cities
process_multiple_cities(target_cities, base_image_folder, base_output_folder)

Using cache found in C:\Users\xiw01/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-12-19 Python-3.10.14 torch-2.4.0 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 
C:\Users\xiw01/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\xiw01/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\xiw01/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\xiw01/